In [ ]:
def calculate_player_metrics_new_system(ranking_df, vbd_df, my_picks=None, drafted_players=None, current_pick=1):
    """Calculate enhanced metrics using new 80% ESPN + 20% ADP probability system"""
    if my_picks is None:
        my_picks = [8, 17, 32, 41, 56, 65, 80, 89]  # Default pick positions
    if drafted_players is None:
    
    # Get next pick logic
    next_pick = None
    picks_to_next = None
    pick_after_next = None
    
    for pick in my_picks:
        if pick >= current_pick:
            next_pick = pick
            picks_to_next = pick - current_pick
            remaining_picks = [p for p in my_picks if p > next_pick]
            pick_after_next = remaining_picks[0] if remaining_picks else None
            break
    
    print(f"NEW SYSTEM - DRAFT POSITION LOGIC:")
    print(f"Current pick: {current_pick}")
    print(f"Your next pick: {next_pick} (in {picks_to_next} picks)")
    print(f"Your pick after that: {pick_after_next}")
    print("=" * 50)
    
    # Filter out drafted players from ranking data
    available_ranking_df = ranking_df[~ranking_df['player_name'].isin(drafted_players)].copy()
    
    # Merge with VBD data for VBD scores
    vbd_lookup = dict(zip(vbd_df['Player'], vbd_df['Custom_VBD']))
    available_ranking_df['vbd_score'] = available_ranking_df['player_name'].map(vbd_lookup)
    available_ranking_df['vbd_score'] = available_ranking_df['vbd_score'].fillna(0)
    
    enhanced_data = []
    
    for _, player in available_ranking_df.iterrows():
        player_name = player['player_name']
        espn_rank = player['espn_rank']
        adp_rank = player['adp_rank']
        vbd_score = player['vbd_score']
        
        # Calculate NEW probabilities using discrete survival method
        prob_gone_by_next = probability_gone_before_next_pick(
            available_ranking_df, player_name, picks_to_next
        ) if picks_to_next > 0 else 0.0
        
        prob_available_at_next = 1 - prob_gone_by_next
        
        # Calculate probability gone by pick after next
        picks_to_after = (pick_after_next - current_pick) if pick_after_next else 0
        prob_gone_by_after = probability_gone_before_next_pick(
            available_ranking_df, player_name, picks_to_after
        ) if picks_to_after > 0 else 0.0
        
        prob_available_at_after = 1 - prob_gone_by_after
        
        # Decision score using VBD × availability probability
        decision_score = vbd_score * prob_available_at_next
        
        # Get immediate next-pick probability for this player
        immediate_pick_probs = compute_pick_probabilities(available_ranking_df)
        player_idx = available_ranking_df.index[available_ranking_df['player_name'] == player_name][0]
        immediate_prob = immediate_pick_probs.loc[player_idx]
        
        # Decision logic
        if prob_available_at_next > 0.8:
            decision_notes = f"SAFE - {prob_available_at_next:.0%} chance available at pick {next_pick}"
        elif prob_available_at_after and prob_available_at_after > 0.7:
            decision_notes = f"WAIT - Target at pick {pick_after_next}"
        elif prob_available_at_next > 0.3:
            decision_notes = f"DRAFT NOW - Only {prob_available_at_next:.0%} chance available later"
        else:
            decision_notes = f"REACH - Must draft at pick {next_pick} to secure"
        
        # Add VBD tier
        if vbd_score >= 100:
            decision_notes = "ELITE - " + decision_notes
        elif vbd_score >= 70:
            decision_notes = "STRONG - " + decision_notes
        elif vbd_score >= 40:
            decision_notes = "SOLID - " + decision_notes
        else:
            decision_notes = "DEPTH - " + decision_notes
        
        enhanced_data.append({
            'player_name': player_name,
            'position': player['position'],
            'position_rank': player['position_rank'],
            'espn_rank': int(espn_rank),
            'adp_rank': int(adp_rank),
            'vbd_score': vbd_score,
            'team': player['team'],
            'bye_week': player['bye_week'],
            'current_pick': current_pick,
            'next_pick': next_pick,
            'picks_to_next': picks_to_next,
            'pick_after_next': pick_after_next,
            'prob_available_at_next': prob_available_at_next,
            'prob_available_at_after': prob_available_at_after,
            'immediate_pick_prob': immediate_prob,
            'decision_score': decision_score,
            'decision_notes': decision_notes
        })
    
    enhanced_df = pd.DataFrame(enhanced_data)
    
    print(f"SUCCESS: Enhanced data calculated for {len(enhanced_df)} available players")
    print(f"Using new 80% ESPN + 20% ADP discrete survival probability system")
    
    return enhanced_df

# Test the new system
my_draft_picks = [8, 17, 32, 41, 56, 65, 80, 89]
current_draft_pick = 1
drafted_players_list = set()  # Empty for now

# Load VBD data for scoring
vbd_data = load_vbd_data()

# Calculate using new system
new_enhanced_df = calculate_player_metrics_new_system(
    ranking_df, vbd_data, 
    my_picks=my_draft_picks, 
    current_pick=current_draft_pick,
    drafted_players=drafted_players_list
)

print(f"\nTop 10 players by ESPN rank using NEW probability system:")
top_new = new_enhanced_df.sort_values('espn_rank').head(10)
print(top_new[['player_name', 'espn_rank', 'adp_rank', 'vbd_score', 'prob_available_at_next', 'decision_notes']].round(3))

NameError: name 'load_vbd_data' is not defined

In [ ]:
def load_and_merge_ranking_data():
    """Load ESPN rankings and external ADP data, merge them for probability calculations"""
    
    # Load ESPN data (80% weight)
    espn_df = pd.read_csv('data/espn_projections_20250814.csv')
    
    # Load FantasyPros ADP data (20% weight)  
    adp_df = pd.read_csv('data/fantasypros_adp_20250815.csv')
    
    print(f"ESPN data: {len(espn_df)} players")
    print(f"ADP data: {len(adp_df)} players")
    
    # Standardize player names for matching
    def clean_name(name):
        return str(name).strip().lower().replace("'", "").replace(".", "")
    
    espn_df['clean_name'] = espn_df['player_name'].apply(clean_name)
    adp_df['clean_name'] = adp_df['PLAYER'].apply(clean_name)
    
    # Merge ESPN with ADP data
    merged_df = espn_df.merge(
        adp_df[['clean_name', 'RANK', 'ADP']], 
        on='clean_name', 
        how='left'
    )
    
    # Fill missing ADP ranks with ESPN rank + some penalty
    merged_df['adp_rank'] = merged_df['RANK'].fillna(merged_df['overall_rank'] + 50)
    merged_df['espn_rank'] = merged_df['overall_rank']
    
    print(f"Merged dataset: {len(merged_df)} players")
    print(f"Players with ADP data: {len(merged_df[merged_df['RANK'].notna()])} players")
    
    return merged_df[['overall_rank', 'position', 'position_rank', 'player_name', 'team', 'salary_value', 'bye_week', 'espn_rank', 'adp_rank']]

def compute_softmax_scores(rank_series, tau=5.0):
    """Convert ranks to softmax scores with temperature tau (lower rank = higher score)"""
    scores = np.exp(-rank_series / tau)
    return scores

def compute_pick_probabilities(available_df, espn_weight=0.8, adp_weight=0.2, tau_espn=5.0, tau_adp=5.0):
    """
    Compute per-player probability of being picked next using weighted softmax over ESPN and ADP ranks
    """
    if len(available_df) == 0:
        return pd.Series(dtype=float)
    
    # Softmax scores for ESPN and ADP ranks
    espn_scores = compute_softmax_scores(available_df['espn_rank'], tau_espn)
    adp_scores = compute_softmax_scores(available_df['adp_rank'], tau_adp)
    
    # Weighted combination (80% ESPN, 20% ADP)
    combined_scores = espn_weight * espn_scores + adp_weight * adp_scores
    
    # Normalize to sum to 1 across available players
    probs = combined_scores / combined_scores.sum()
    return probs

def probability_gone_before_next_pick(available_df, player_name, picks_until_next_turn):
    """
    Compute probability a player is gone before your next turn using discrete survival calculation
    """
    if picks_until_next_turn <= 0:
        return 0.0
    
    survival_prob = 1.0
    current_available = available_df.copy()
    
    # Simulate each pick until our next turn
    for pick_step in range(picks_until_next_turn):
        if len(current_available) == 0:
            break
            
        # Get probabilities for current available players
        pick_probs = compute_pick_probabilities(current_available)
        
        # Find probability our target player gets picked this round
        player_mask = current_available['player_name'] == player_name
        if not player_mask.any():
            # Player already gone
            break
            
        p_pick_now = pick_probs[player_mask].iloc[0] if player_mask.any() else 0.0
        
        # Update survival probability
        survival_prob *= (1 - p_pick_now)
        
        # For simulation: remove the most likely pick (simplified approach)
        most_likely_idx = pick_probs.idxmax()
        current_available = current_available.drop(most_likely_idx)
    
    prob_gone = 1 - survival_prob
    return min(1.0, max(0.0, prob_gone))

# Load the new merged ranking data
ranking_df = load_and_merge_ranking_data()
print(f"\nSample of merged ranking data:")
print(ranking_df[['player_name', 'espn_rank', 'adp_rank']].head(10))

# Fantasy Football VBD Ranking Dashboard

Clean, minimal, interactive sortable table showing VBD rankings with probability intelligence.

In [39]:
import pandas as pd
import numpy as np
import plotly.express as px
import plotly.graph_objects as go
from plotly.subplots import make_subplots
from scipy import stats
import ipywidgets as widgets
from IPython.display import display, clear_output, HTML
from IPython.core.display import display_html
import base64
from io import BytesIO
import matplotlib.pyplot as plt
import matplotlib
matplotlib.use('Agg')
import warnings
warnings.filterwarnings('ignore')

## 1. Core Calculation Engine (Preserved from Original)

In [40]:
def load_espn_data():
    """Load ESPN projections CSV data"""
    df = pd.read_csv('data/espn_projections_20250814.csv')
    return df

def load_vbd_data():
    """Load custom VBD rankings"""
    return pd.read_csv('draft_cheat_sheet.csv')

def merge_vbd_with_espn():
    """Merge VBD data with ESPN projections, using VBD rankings as primary"""
    vbd_df = load_vbd_data()
    espn_df = load_espn_data()
    
    print(f"VBD data loaded: {len(vbd_df)} players")
    print(f"ESPN data loaded: {len(espn_df)} players")
    
    # Start with VBD data as the foundation
    final_df = vbd_df.copy()
    
    # Standardize column names to match ESPN format
    final_df['player_name'] = final_df['Player']
    final_df['position'] = final_df['Position'] 
    final_df['overall_rank'] = final_df['Draft_Rank']
    final_df['salary_value'] = final_df['Custom_VBD']  # VBD values replace ESPN salary
    final_df['team'] = final_df['Team']
    final_df['bye_week'] = final_df['Bye']
    
    # Create position_rank from VBD data  
    position_ranks = []
    for _, row in final_df.iterrows():
        pos = row['position']
        # Count how many of this position come before this player in VBD ranking
        pos_rank = len(vbd_df[(vbd_df['Position'] == pos) & (vbd_df['Draft_Rank'] < row['overall_rank'])]) + 1
        position_ranks.append(f"{pos}{pos_rank}")
    
    final_df['position_rank'] = position_ranks
    
    # Keep only the ESPN-format columns
    final_df = final_df[['overall_rank', 'position', 'position_rank', 'player_name', 'team', 'salary_value', 'bye_week']].copy()
    
    # Fill any remaining NaN values
    final_df = final_df.fillna(0)
    
    print(f"Successfully merged {len(final_df)} players from VBD data")
    print(f"Top 5 by VBD score:")
    print(final_df.head()[['player_name', 'position', 'overall_rank', 'salary_value']])
    
    # Debug: Show a few more to verify the merge worked
    print(f"\nVerification - All players using VBD values:")
    print(final_df[['player_name', 'overall_rank', 'salary_value']].head(10))
    
    return final_df

def calculate_availability(player_rank, draft_position, std_dev=3):
    """Calculate probability that a player is available at a given draft position"""
    if draft_position <= 0:
        return 1.0
    # Probability = 1 - CDF(draft_position, mean=player_rank, std=std_dev)
    prob = 1 - stats.norm.cdf(draft_position, loc=player_rank, scale=std_dev)
    return max(0, min(1, prob))

# Load the merged VBD + ESPN data
df = merge_vbd_with_espn()
print(f"\nLoaded {len(df)} players with VBD integration")
print("Sample of enhanced data:")
print(df.head())

VBD data loaded: 50 players
ESPN data loaded: 300 players
Successfully merged 50 players from VBD data
Top 5 by VBD score:
           player_name position  overall_rank  salary_value
0       Saquon Barkley       RB             1         131.3
1       Bijan Robinson       RB             2         127.3
2         Jahmyr Gibbs       RB             3         125.0
3        Ja'Marr Chase       WR             4         120.6
4  Christian McCaffrey       RB             5          96.9

Verification - All players using VBD values:
           player_name  overall_rank  salary_value
0       Saquon Barkley             1         131.3
1       Bijan Robinson             2         127.3
2         Jahmyr Gibbs             3         125.0
3        Ja'Marr Chase             4         120.6
4  Christian McCaffrey             5          96.9
5        Derrick Henry             6          95.8
6        De'Von Achane             7          92.7
7        Lamar Jackson             8          91.7
8     Justin

## 2. Enhanced Data Processing for Dashboard

In [41]:
def calculate_player_metrics(df, my_picks=None, drafted_players=None, current_pick=1):
    """Calculate enhanced metrics for each player with dynamic draft position logic"""
    if my_picks is None:
        my_picks = [8, 17, 32, 41, 56, 65, 80, 89]  # Default pick positions
    if drafted_players is None:
        drafted_players = set()
    
    # Calculate dynamic draft positions
    next_pick = None
    picks_to_next = None
    pick_after_next = None
    
    for pick in my_picks:
        if pick >= current_pick:
            next_pick = pick
            picks_to_next = pick - current_pick
            # Find pick after next
            remaining_picks = [p for p in my_picks if p > next_pick]
            pick_after_next = remaining_picks[0] if remaining_picks else None
            break
    
    print(f"DRAFT POSITION LOGIC:")
    print(f"Current pick: {current_pick}")
    print(f"Your next pick: {next_pick} (in {picks_to_next} picks)")
    print(f"Your pick after that: {pick_after_next}")
    print("=" * 50)
    
    enhanced_data = []
    
    # Calculate position scarcity data using VBD values
    position_counts = {}
    for pos in ['QB', 'RB', 'WR', 'TE']:
        pos_players = df[df['position'] == pos]
        # Count players above median VBD threshold (Custom_VBD >= 40)
        elite_count = len(pos_players[pos_players['salary_value'] >= 40])
        position_counts[pos] = elite_count
    
    for _, player in df.iterrows():
        if player['player_name'] in drafted_players:
            continue  # Skip drafted players
            
        # Use Draft_Rank instead of ESPN overall_rank for availability calculations
        player_rank = player['overall_rank']  # This is now Draft_Rank from VBD data
        custom_vbd = player['salary_value']   # This is now Custom_VBD from VBD data
        position = player['position']
        
        # Calculate probabilities at dynamic picks
        prob_at_next_pick = calculate_availability(player_rank, next_pick) if next_pick else 0.0
        prob_at_pick_after = calculate_availability(player_rank, pick_after_next) if pick_after_next else 0.0
        
        # Calculate Decision Score using Custom_VBD × Probability at next pick
        decision_score = custom_vbd * prob_at_next_pick
        
        # Calculate median pick using VBD Draft_Rank (50% probability point)
        median_pick = player_rank
        
        # Calculate 10th and 90th percentile picks for range using VBD ranking
        p10_pick = max(1, stats.norm.ppf(0.9, loc=player_rank, scale=3))
        p90_pick = stats.norm.ppf(0.1, loc=player_rank, scale=3)
        
        # Calculate opportunity cost (next best VBD value at position if wait)
        pos_players = df[df['position'] == position]
        # Sort by VBD ranking (overall_rank which is now Draft_Rank)
        better_players = pos_players[pos_players['overall_rank'] > player_rank].sort_values('overall_rank')
        if len(better_players) > 0:
            next_best_vbd = better_players.iloc[0]['salary_value'] if len(better_players) > 0 else 0
            opportunity_cost = max(0, custom_vbd - next_best_vbd)  # Cost of waiting vs drafting now
        else:
            opportunity_cost = 0
        
        # Positional scarcity calculation using VBD position ranking
        pos_rank_num = int(player['position_rank'].replace(position, '')) if player['position_rank'] else 999
        if position in position_counts:
            remaining_elite = max(0, position_counts[position] - pos_rank_num + 1)
            scarcity_text = f"{position}{pos_rank_num} ({remaining_elite} of {position_counts[position]} left)"
        else:
            scarcity_text = f"{position}{pos_rank_num}"
        
        # Generate decision notes with strategic guidance
        decision_notes = ""
        if prob_at_next_pick > 0.8:
            decision_notes = f"SAFE - Available at pick {next_pick}"
        elif prob_at_pick_after and prob_at_pick_after > 0.7:
            decision_notes = f"WAIT - Target at pick {pick_after_next}"
        elif prob_at_next_pick > 0.3:
            decision_notes = f"DRAFT NOW - Risky to wait"
        else:
            decision_notes = f"REACH - Must draft at pick {next_pick} to secure"
        
        # Add value consideration
        if custom_vbd >= 100:
            decision_notes = "ELITE - " + decision_notes
        elif custom_vbd >= 70:
            decision_notes = "STRONG - " + decision_notes
        elif custom_vbd >= 40:
            decision_notes = "SOLID - " + decision_notes
        else:
            decision_notes = "DEPTH - " + decision_notes
        
        # Generate sparkline data using VBD Draft_Rank (availability across next 16 picks)
        sparkline_picks = list(range(max(1, player_rank - 5), player_rank + 12))
        sparkline_probs = [calculate_availability(player_rank, pick) for pick in sparkline_picks]
        
        enhanced_data.append({
            'player_name': player['player_name'],
            'position': player['position'],
            'position_rank': player['position_rank'],
            'overall_rank': player['overall_rank'],  # This is now VBD Draft_Rank
            'team': player['team'],
            'salary_value': player['salary_value'],  # This is now Custom_VBD
            'bye_week': player['bye_week'],
            'current_pick': current_pick,
            'next_pick': next_pick,
            'picks_to_next': picks_to_next,
            'pick_after_next': pick_after_next,
            'prob_at_next_pick': prob_at_next_pick,
            'prob_at_pick_after': prob_at_pick_after,
            'decision_score': decision_score,  # Now uses Custom_VBD × Probability at next pick
            'opportunity_cost': opportunity_cost,
            'scarcity_badge': scarcity_text,
            'decision_notes': decision_notes,
            'median_pick': median_pick,
            'p10_pick': int(p10_pick),
            'p90_pick': int(max(1, p90_pick)),
            'pick_range': f"{int(max(1, p90_pick))}-{int(p10_pick)}",
            'sparkline_picks': sparkline_picks,
            'sparkline_probs': sparkline_probs
        })
    
    enhanced_df = pd.DataFrame(enhanced_data)
    
    print(f"SUCCESS: All {len(enhanced_df)} players now use dynamic draft position logic")
    print(f"Probabilities calculated for next pick ({next_pick}) and pick after ({pick_after_next})")
    
    return enhanced_df

# Calculate metrics with dynamic draft position logic
my_draft_picks = [8, 17, 32, 41, 56, 65, 80, 89]  # 8-team league, pick 8
current_draft_pick = 1  # Current pick in the draft

enhanced_df = calculate_player_metrics(df, my_picks=my_draft_picks, current_pick=current_draft_pick)
print(f"\nEnhanced data for {len(enhanced_df)} available players with dynamic draft logic")
print("\nTop 10 players by VBD Rank:")
top_vbd = enhanced_df.sort_values('overall_rank').head(10)
print(top_vbd[['player_name', 'position', 'overall_rank', 'salary_value', 'prob_at_next_pick', 'prob_at_pick_after', 'decision_notes']].round(2))

DRAFT POSITION LOGIC:
Current pick: 1
Your next pick: 8 (in 7 picks)
Your pick after that: 17
SUCCESS: All 50 players now use dynamic draft position logic
Probabilities calculated for next pick (8) and pick after (17)

Enhanced data for 50 available players with dynamic draft logic

Top 10 players by VBD Rank:
           player_name position  overall_rank  salary_value  \
0       Saquon Barkley       RB             1         131.3   
1       Bijan Robinson       RB             2         127.3   
2         Jahmyr Gibbs       RB             3         125.0   
3        Ja'Marr Chase       WR             4         120.6   
4  Christian McCaffrey       RB             5          96.9   
5        Derrick Henry       RB             6          95.8   
6        De'Von Achane       RB             7          92.7   
7        Lamar Jackson       QB             8          91.7   
8     Justin Jefferson       WR             9          88.3   
9           Josh Allen       QB            10          88.

In [44]:
sort_table('raw', 'salary_value', False)

🔄 Sorting raw table by salary_value (desc)
📊 RAW DATA TABLE - Current Pick: 1 | Next Pick: 8 (in 7 picks)
Sorted by: salary_value (descending)


,VBD Rank,Player,Pos,Team,VBD Score,Next Pick (in 7)%,Pick After (in 9)%,Decision Score,Opp Cost,Median Pick,Range,Bye,Scarcity,Decision Notes
0,1,Saquon Barkley,RB,PHI,131.3,1.0,0.0,1.3,4.0,1,1-4,9,RB1 (16 of 16 left),ELITE - REACH - Must draft at pick 8 to secure
1,2,Bijan Robinson,RB,ATL,127.3,2.3,0.0,2.9,2.3,2,1-5,5,RB2 (15 of 16 left),ELITE - REACH - Must draft at pick 8 to secure
2,3,Jahmyr Gibbs,RB,DET,125.0,4.8,0.0,6.0,28.1,3,1-6,8,RB3 (14 of 16 left),ELITE - REACH - Must draft at pick 8 to secure
3,4,Ja'Marr Chase,WR,CIN,120.6,9.1,0.0,11.0,32.3,4,1-7,10,WR1 (9 of 9 left),ELITE - REACH - Must draft at pick 8 to secure
4,5,Christian McCaffrey,RB,SF,96.9,15.9,0.0,15.4,1.1,5,1-8,14,RB4 (13 of 16 left),STRONG - REACH - Must draft at pick 8 to secure
5,6,Derrick Henry,RB,BAL,95.8,25.2,0.0,24.2,3.1,6,2-9,7,RB5 (12 of 16 left),STRONG - REACH - Must draft at pick 8 to secure
6,7,De'Von Achane,RB,MIA,92.7,36.9,0.0,34.2,12.9,7,3-10,12,RB6 (11 of 16 left),STRONG - DRAFT NOW - Risky to wait
7,8,Lamar Jackson,QB,BAL,91.7,50.0,0.1,45.9,3.5,8,4-11,7,QB1 (5 of 5 left),STRONG - DRAFT NOW - Risky to wait
8,9,Justin Jefferson,WR,MIN,88.3,63.1,0.4,55.7,13.9,9,5-12,6,WR2 (8 of 9 left),STRONG - DRAFT NOW - Risky to wait
9,10,Josh Allen,QB,BUF,88.2,74.8,1.0,65.9,18.7,10,6-13,7,QB2 (4 of 5 left),STRONG - DRAFT NOW - Risky to wait


In [42]:
def create_sparkline(probabilities, width=60, height=20):
    """Create a sparkline SVG image from probability data"""
    if not probabilities or len(probabilities) < 2:
        return "data:image/svg+xml;base64," + base64.b64encode(
            f'<svg width="{width}" height="{height}"></svg>'.encode()
        ).decode()
    
    # Normalize probabilities to SVG coordinates
    max_prob = max(probabilities)
    min_prob = min(probabilities)
    if max_prob == min_prob:
        max_prob += 0.1  # Avoid division by zero
    
    # Create SVG path
    svg_points = []
    for i, prob in enumerate(probabilities):
        x = (i / (len(probabilities) - 1)) * width
        y = height - ((prob - min_prob) / (max_prob - min_prob)) * height
        svg_points.append(f"{x},{y}")
    
    path_data = "M" + "L".join(svg_points)
    
    # Create SVG with sparkline
    svg = f'''<svg width="{width}" height="{height}" xmlns="http://www.w3.org/2000/svg">
        <path d="{path_data}" stroke="#440154" stroke-width="1.5" fill="none"/>
        <circle cx="{svg_points[0].split(',')[0]}" cy="{svg_points[0].split(',')[1]}" r="1.5" fill="#440154"/>
        <circle cx="{svg_points[-1].split(',')[0]}" cy="{svg_points[-1].split(',')[1]}" r="1.5" fill="#440154"/>
    </svg>'''
    
    # Return as data URI
    return "data:image/svg+xml;base64," + base64.b64encode(svg.encode()).decode()

def create_pick_distribution_bar(p10_pick, median_pick, p90_pick, width=80, height=16):
    """Create an SVG distribution bar showing pick range with percentiles"""
    # Ensure valid range
    if p90_pick >= p10_pick:
        p90_pick, p10_pick = p10_pick, p90_pick
    
    total_range = max(p10_pick - p90_pick, 1)
    
    # Calculate positions on the bar (0 to width)
    bar_start = 0
    bar_end = width
    median_pos = ((median_pick - p90_pick) / total_range) * width if total_range > 0 else width / 2
    median_pos = max(0, min(width, median_pos))  # Clamp to bar width
    
    # Create SVG bar
    svg = f'''<svg width="{width}" height="{height}" xmlns="http://www.w3.org/2000/svg">
        <!-- Background bar (full range) -->
        <rect x="0" y="6" width="{width}" height="4" fill="#e0e0e0" rx="2"/>
        <!-- Range bar (10th to 90th percentile) -->
        <rect x="{bar_start}" y="6" width="{bar_end}" height="4" fill="#31688e" rx="2"/>
        <!-- Median marker -->
        <circle cx="{median_pos}" cy="8" r="3" fill="#440154" stroke="white" stroke-width="1"/>
        <!-- Labels -->
        <text x="0" y="15" font-size="8" fill="#666" text-anchor="start">{p90_pick}</text>
        <text x="{width}" y="15" font-size="8" fill="#666" text-anchor="end">{p10_pick}</text>
    </svg>'''
    
    return svg

In [43]:
def create_raw_data_table(enhanced_df, sort_by='overall_rank', ascending=True, limit=50):
    """Table 1: Simple raw data table with all columns - VS Code compatible"""
    import pandas as pd
    from IPython.display import display, HTML
    
    # Sort the data
    df_display = enhanced_df.sort_values(sort_by, ascending=ascending).head(limit).copy()
    
    # Get dynamic pick information
    next_pick = df_display['next_pick'].iloc[0] if len(df_display) > 0 else "N/A"
    pick_after = df_display['pick_after_next'].iloc[0] if len(df_display) > 0 else None
    picks_to_next = df_display['picks_to_next'].iloc[0] if len(df_display) > 0 else "N/A"
    current_pick = df_display['current_pick'].iloc[0] if len(df_display) > 0 else 1
    
    # Calculate picks between next and pick after
    picks_between = (pick_after - next_pick) if (pick_after and next_pick) else "N/A"
    
    # Create clean table with minimal decimal places
    table_df = pd.DataFrame({
        'VBD Rank': df_display['overall_rank'].astype(int),
        'Player': df_display['player_name'],
        'Pos': df_display['position'], 
        'Team': df_display['team'],
        'VBD Score': df_display['salary_value'].apply(lambda x: int(x) if x == int(x) else round(x, 1)),
        f'Next Pick (in {picks_to_next})%': (df_display['prob_at_next_pick'] * 100).apply(lambda x: int(x) if x == int(x) else round(x, 1)),
        f'Pick After (in {picks_between})%': (df_display['prob_at_pick_after'] * 100).apply(lambda x: int(x) if x == int(x) else round(x, 1)),
        'Decision Score': df_display['decision_score'].apply(lambda x: int(x) if x == int(x) else round(x, 1)),
        'Opp Cost': df_display['opportunity_cost'].apply(lambda x: int(x) if x == int(x) else round(x, 1)),
        'Median Pick': df_display['median_pick'].astype(int),
        'Range': df_display['pick_range'],
        'Bye': df_display['bye_week'].astype(int),
        'Scarcity': df_display['scarcity_badge'],
        'Decision Notes': df_display['decision_notes']
    })
    
    # Display header
    print(f"📊 RAW DATA TABLE - Current Pick: {current_pick} | Next Pick: {next_pick} (in {picks_to_next} picks)")
    print(f"Sorted by: {sort_by} ({'ascending' if ascending else 'descending'})")
    print("="*100)
    
    return table_df

def create_strategic_insights_table(enhanced_df, sort_by='overall_rank', ascending=True, limit=25):
    """Table 2: Strategic insights with key columns - VS Code compatible"""
    import pandas as pd
    from IPython.display import display, HTML
    
    # Sort the data
    df_display = enhanced_df.sort_values(sort_by, ascending=ascending).head(limit).copy()
    
    # Get dynamic pick information
    next_pick = df_display['next_pick'].iloc[0] if len(df_display) > 0 else "N/A"
    pick_after = df_display['pick_after_next'].iloc[0] if len(df_display) > 0 else None
    picks_to_next = df_display['picks_to_next'].iloc[0] if len(df_display) > 0 else "N/A"
    current_pick = df_display['current_pick'].iloc[0] if len(df_display) > 0 else 1
    
    # Calculate picks between next and pick after
    picks_between = (pick_after - next_pick) if (pick_after and next_pick) else "N/A"
    
    # Create strategic table with minimal decimals
    table_df = pd.DataFrame({
        'VBD Rank': df_display['overall_rank'].astype(int),
        'Player': df_display['player_name'],
        'VBD Score': df_display['salary_value'].apply(lambda x: int(x) if x == int(x) else round(x, 1)),
        f'Next ({picks_to_next})%': (df_display['prob_at_next_pick'] * 100).apply(lambda x: int(x) if x == int(x) else round(x, 1)),
        f'After ({picks_between})%': (df_display['prob_at_pick_after'] * 100).apply(lambda x: int(x) if x == int(x) else round(x, 1)),
        'Decision Notes': df_display['decision_notes']
    })
    
    print(f"🎯 STRATEGIC INSIGHTS - Current Pick: {current_pick} | Next Pick: {next_pick} (in {picks_to_next} picks)")
    print(f"Sorted by: {sort_by} ({'ascending' if ascending else 'descending'})")
    print("="*100)
    
    return table_df

def sort_table(table_type='raw', sort_by='overall_rank', ascending=True):
    """Easy sorting function for VS Code - call this to re-sort tables"""
    print(f"🔄 Sorting {table_type} table by {sort_by} ({'asc' if ascending else 'desc'})")
    if table_type == 'raw':
        return create_raw_data_table(enhanced_df, sort_by=sort_by, ascending=ascending)
    else:
        return create_strategic_insights_table(enhanced_df, sort_by=sort_by, ascending=ascending)

# Display both tables with default VBD ranking
print("🏈 FANTASY FOOTBALL VBD TABLES")
print("=" * 50)
print("💡 SORTING INSTRUCTIONS:")
print("   sort_table('raw', 'column_name', True/False)")
print("   sort_table('strategic', 'column_name', True/False)")
print()
print("📋 AVAILABLE COLUMNS:")
print("   'overall_rank' (VBD Rank), 'salary_value' (VBD Score), 'prob_at_next_pick' (Next Pick %)")
print("   'decision_score', 'opportunity_cost', 'player_name', 'median_pick'")
print()
print("🔥 QUICK EXAMPLES:")
print("   sort_table('raw', 'salary_value', False)  # Highest VBD Score first")
print("   sort_table('strategic', 'prob_at_next_pick', False)  # Best availability first")
print("   sort_table('raw', 'player_name', True)  # Alphabetical by player")
print()

# Show both tables
raw_table = create_raw_data_table(enhanced_df)
print("\n")
strategic_table = create_strategic_insights_table(enhanced_df)

display(raw_table)
print("\n" + "="*50 + "\n")
display(strategic_table)

🏈 FANTASY FOOTBALL VBD TABLES
💡 SORTING INSTRUCTIONS:
   sort_table('raw', 'column_name', True/False)
   sort_table('strategic', 'column_name', True/False)

📋 AVAILABLE COLUMNS:
   'overall_rank' (VBD Rank), 'salary_value' (VBD Score), 'prob_at_next_pick' (Next Pick %)
   'decision_score', 'opportunity_cost', 'player_name', 'median_pick'

🔥 QUICK EXAMPLES:
   sort_table('raw', 'salary_value', False)  # Highest VBD Score first
   sort_table('strategic', 'prob_at_next_pick', False)  # Best availability first
   sort_table('raw', 'player_name', True)  # Alphabetical by player

📊 RAW DATA TABLE - Current Pick: 1 | Next Pick: 8 (in 7 picks)
Sorted by: overall_rank (ascending)


🎯 STRATEGIC INSIGHTS - Current Pick: 1 | Next Pick: 8 (in 7 picks)
Sorted by: overall_rank (ascending)


,VBD Rank,Player,Pos,Team,VBD Score,Next Pick (in 7)%,Pick After (in 9)%,Decision Score,Opp Cost,Median Pick,Range,Bye,Scarcity,Decision Notes
0,1,Saquon Barkley,RB,PHI,131.3,1.0,0.0,1.3,4.0,1,1-4,9,RB1 (16 of 16 left),ELITE - REACH - Must draft at pick 8 to secure
1,2,Bijan Robinson,RB,ATL,127.3,2.3,0.0,2.9,2.3,2,1-5,5,RB2 (15 of 16 left),ELITE - REACH - Must draft at pick 8 to secure
2,3,Jahmyr Gibbs,RB,DET,125.0,4.8,0.0,6.0,28.1,3,1-6,8,RB3 (14 of 16 left),ELITE - REACH - Must draft at pick 8 to secure
3,4,Ja'Marr Chase,WR,CIN,120.6,9.1,0.0,11.0,32.3,4,1-7,10,WR1 (9 of 9 left),ELITE - REACH - Must draft at pick 8 to secure
4,5,Christian McCaffrey,RB,SF,96.9,15.9,0.0,15.4,1.1,5,1-8,14,RB4 (13 of 16 left),STRONG - REACH - Must draft at pick 8 to secure
5,6,Derrick Henry,RB,BAL,95.8,25.2,0.0,24.2,3.1,6,2-9,7,RB5 (12 of 16 left),STRONG - REACH - Must draft at pick 8 to secure
6,7,De'Von Achane,RB,MIA,92.7,36.9,0.0,34.2,12.9,7,3-10,12,RB6 (11 of 16 left),STRONG - DRAFT NOW - Risky to wait
7,8,Lamar Jackson,QB,BAL,91.7,50.0,0.1,45.9,3.5,8,4-11,7,QB1 (5 of 5 left),STRONG - DRAFT NOW - Risky to wait
8,9,Justin Jefferson,WR,MIN,88.3,63.1,0.4,55.7,13.9,9,5-12,6,WR2 (8 of 9 left),STRONG - DRAFT NOW - Risky to wait
9,10,Josh Allen,QB,BUF,88.2,74.8,1.0,65.9,18.7,10,6-13,7,QB2 (4 of 5 left),STRONG - DRAFT NOW - Risky to wait


,VBD Rank,Player,VBD Score,Next (7)%,After (9)%,Decision Notes
0,1,Saquon Barkley,131.3,1.0,0.0,ELITE - REACH - Must draft at pick 8 to secure
1,2,Bijan Robinson,127.3,2.3,0.0,ELITE - REACH - Must draft at pick 8 to secure
2,3,Jahmyr Gibbs,125.0,4.8,0.0,ELITE - REACH - Must draft at pick 8 to secure
3,4,Ja'Marr Chase,120.6,9.1,0.0,ELITE - REACH - Must draft at pick 8 to secure
4,5,Christian McCaffrey,96.9,15.9,0.0,STRONG - REACH - Must draft at pick 8 to secure
5,6,Derrick Henry,95.8,25.2,0.0,STRONG - REACH - Must draft at pick 8 to secure
6,7,De'Von Achane,92.7,36.9,0.0,STRONG - DRAFT NOW - Risky to wait
7,8,Lamar Jackson,91.7,50.0,0.1,STRONG - DRAFT NOW - Risky to wait
8,9,Justin Jefferson,88.3,63.1,0.4,STRONG - DRAFT NOW - Risky to wait
9,10,Josh Allen,88.2,74.8,1.0,STRONG - DRAFT NOW - Risky to wait


## 3. New Dynamic Probability System (80% ESPN + 20% ADP)